<a href="https://colab.research.google.com/github/Approach-Analytics/Emotion-Classifier/blob/main/Train_Fear_Distil_TF_June3%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trying out a pre-trained language model

The issues that we are going to need to address are:


1.   What are compute requirements...
2.   How does the whole tokenization / embedding thing work...
3.   What training set(s) to use?
4.   What model to use that doesn't mess with our inference?


Resources:
https://www.tensorflow.org/text/tutorials/classify_text_with_bert



#Import libraries

In [1]:
!pip install tensorflow
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification

In [3]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import pickle

In [4]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


In [5]:
#Dec 7, 2022: Not sure what this piece of code is used for...

import warnings
warnings.filterwarnings('ignore')

In [6]:
#Setting column options

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 100)

#Supressing the scientific notation

pd.set_option('display.float_format', lambda x: '%.3f' % x)

#Setting up TPU If Necessary

#Import the dataset

In [7]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#Trying new dataset

#path = '/content/drive/MyDrive/Sean/Emoclass/Emotion datasets/Fear_Gut55K_3.2mil_June8.csv'

path = '/content/drive/MyDrive/Sean/Emoclass/Emotion datasets/Fear_Gut55K_1.3mil_June8.csv'
df = pd.read_csv(path)

In [9]:
df.shape

(1254032, 3)

In [10]:
df['emotion2'].value_counts()

fear       444444
calm       245184
concern    164099
dread      161290
anxiety    127904
worry      111111
Name: emotion2, dtype: int64

In [11]:
df.head(25)

,text,emotion1,emotion2
0,of the events of the day and paying a tribute ...,fear,fear
1,long alone as soon as the great folks had fair...,fear,fear
2,the politics of rome and not against its relig...,fright,fear
3,wave to the people i know i d best go where th...,fear,fear
4,himself at besançon where he will set up as a ...,fear,fear
5,mr arbuthnot awaiting the physician morning re...,fear,fear
6,covering everyone sat still until the disentan...,afraid,fear
7,saved from becoming fat sick mean wheezy or dy...,fearful,fear
8,enough to receive him but now he had met anton...,frightened,fear
9,toward the dark fields the arno formed lagoons...,fright,fear


In [12]:
# Create a 50% sample of the DataFrame
#df = df.sample(frac=0.5, random_state=42)


In [13]:
#Relabelling a column...

df.rename(columns={"emotion2": "label"},inplace =True)

#Import the emotion axis

In [14]:
#Defining the emotion axis - note the use of the ordered dict here...

from collections import OrderedDict

emo_axis = OrderedDict([
    ("dread", ['panic','dread','horror','horrified','horrifying','terror','terrified','terrifying']),
    ("fear", ["fear", "fearful", "fright","frightening","afraid","frightful","frightfully","frightened",'scared','scary','scare']),
    ("anxiety", ["anxious","anxiety",'angst','anxiousness']),
    ("worry", ["worry", "worried","worrying","worries"]),
    ("concern", ["concern", "concerning","concerned","concerns"]),
    ("calm", ["calm","peaceful","serene","serenity","untroubled", "content", "contented", "composed", "tranquil"])
])


In [15]:
# Assign numerical labels to emotion categories in reversed order
labels = list(emo_axis.keys())
label_mapping = {label: i for i, label in enumerate(reversed(labels))}


In [16]:
#check the label mapping

label_mapping

{'calm': 0, 'concern': 1, 'worry': 2, 'anxiety': 3, 'fear': 4, 'dread': 5}

#Remove key emotion words

In [17]:
#check the right label is used...

df['emotion1'].value_counts()

fear           199472
afraid         119115
anxious         78652
concerning      67898
content         59953
calm            59771
worry           56413
composed        55708
concerned       55382
terror          53554
anxiety         44025
frightened      43387
horror          40515
worried         34254
dread           32310
fearful         29604
concern         27497
peaceful        23695
contented       19097
frightful       15576
terrified       14156
fright          13429
concerns        13322
scared          12533
worrying        12360
panic           11693
serene          11234
tranquil         9000
worries          8084
scare            5810
angst            5170
serenity         5127
horrified        5123
terrifying       3575
frightfully      3318
frightening      1990
untroubled       1599
horrifying        364
scary             210
anxiousness        57
Name: emotion1, dtype: int64

In [18]:
#Generate all the unique emotion words that then get replaced...

a = df['emotion1'].unique().tolist()

#The list a is our list of variable responses from the dataset...
keyword = "emotions"
words = a
for j in words:
  df['text'] = df['text'].str.replace(j,keyword)

##Splitting into train, validate and test

In [19]:
#This outputs 3 different dataframes... originally 0.6 and 0.8

train, validate, test = np.split(df.sample(frac=1, random_state=42),
                       [int(.8*len(df)), int(.9*len(df))])

##Preparing the Labels

In [20]:
#Converting the pandas dataframe into a list of labels...
#We may consider puting this into a function...

trainlabel=train['label'].tolist()
vallabel=validate['label'].tolist()
testlabel=test['label'].tolist()

In [21]:
#Test the label mapping again - dyslexic after all...

label_mapping

{'calm': 0, 'concern': 1, 'worry': 2, 'anxiety': 3, 'fear': 4, 'dread': 5}

In [22]:
# Convert the train, validation, and test labels using label_mapping
train_labels = [label_mapping[label] for label in trainlabel]
val_labels = [label_mapping[label] for label in vallabel]
test_labels = [label_mapping[label] for label in testlabel]


In [23]:
#Convert labels to Tensors (int32)

train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_labels, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

#Training and Tokenizing Tweets

---



In [24]:
#Converting the different pandas dataframes into a list of text fields...
#Choice of variables: truncated_text, text_minus1, 'filtered_text',''filtered_text_minus1'

traintext=train['text'].tolist()
valtext=validate['text'].tolist()
testtext=test['text'].tolist()

##Load the Tokenizer and Set Batch Size

V100 High RAM runs on batch size of 30 (14.9/16Gb RAM)

In [25]:
import numpy as np
import tensorflow as tf
from transformers import DistilBertTokenizerFast

In [26]:
#Load the tokenizer...

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [27]:
#Setting the batch size
#Batch size for T4 High ram is: 25 (1ith 1.3 million)
#Batch size for V100 high RAM is: 30
#Batch size A100 HIgh ram is 80

batch_size = 25

In [28]:

def tokenize_and_map(text, label):
    text = text.numpy().decode('utf-8')
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512)
    return encoding['input_ids'], encoding['attention_mask'], label

def tf_tokenize_and_map(text, label):
    input_ids, attention_mask, label = tf.py_function(tokenize_and_map, inp=[text, label], Tout=(tf.int32, tf.int32, tf.int32))
    input_ids.set_shape([None])
    attention_mask.set_shape([None])
    label.set_shape([])
    return {'input_ids': input_ids, 'attention_mask': attention_mask}, label

# Create a Dataset for the train data
train_dataset = tf.data.Dataset.from_tensor_slices((traintext, train_labels))
train_dataset = train_dataset.map(tf_tokenize_and_map)
train_dataset = train_dataset.batch(batch_size)

# Repeat the same steps for validation and test data
val_dataset = tf.data.Dataset.from_tensor_slices((valtext, val_labels))
val_dataset = val_dataset.map(tf_tokenize_and_map)
val_dataset = val_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((testtext, test_labels))
test_dataset = test_dataset.map(tf_tokenize_and_map)
test_dataset = test_dataset.batch(batch_size)


#Compiling and Fitting the Model

In [29]:
#Load the model

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [30]:
# Define the ModelCheckpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Sean/Emoclass/checkpoint_fear_3.3_June8,2023',  # Path to save the model
    save_weights_only=True,       # Save only the model weights
    save_freq='epoch'             # Save the model after each epoch
)



In [31]:
#Experimenting with changing the learning rate...

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
# Train the model with checkpoint callback
model.fit(
    train_dataset.shuffle(1000),
    epochs=3,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)

Epoch 1/3
  420/40129 [..............................] - ETA: 14:01:57 - loss: 1.1095 - accuracy: 0.5878

##TRying with Tensorflow

In [ ]:
import os
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
    model = create_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )

model.fit(
    train_dataset.shuffle(1000),
    epochs=3,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)


#Evaluate the model

In [ ]:
model.evaluate(test_dataset.batch(16))

In [ ]:
#Make predictions on test data

predictions = model.predict(test_dataset.batch(16))


In [ ]:



#Convert prediction score to predicted classes

predicted_classes = tf.argmax(predictions.logits, axis=-1)

predicted_classes = tf.cast(tf.argmax(predictions.logits, axis=-1), dtype=tf.int32)

#Calculate absolute difference

abs_diffs = tf.abs(predicted_classes - test_labels)

In [ ]:
#Create histogram of absolute differences

import matplotlib.pyplot as plt

plt.hist(abs_diffs.numpy(), bins=11)
plt.xlabel('Absolute difference')
plt.ylabel('Frequency')
plt.title('Distribution of absolute differences between predicted and true classes')
plt.show()


#Save the tokenizer and the model

In [ ]:
#path = /content/drive/MyDrive/Sean/Emoclass

# Save the model
model.save_pretrained('/content/drive/MyDrive/Sean/Emoclass/Model_Fear_Twitter_100K_Distill_May20,2023')

# Save the tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/Sean/Emoclass/Distill_Tokenizer_May20')
